In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline


In [ ]:
df = pd.read_csv("processed.csv",encoding="utf-8",lineterminator='\n')

In [ ]:
df

In [ ]:
X = df.content
Y = df.label
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.30)

In [ ]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)


In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model


In [ ]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])


In [ ]:
model.fit(sequences_matrix,Y_train,batch_size=1024,epochs=5,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='accuracy',min_delta=0.0001)])


In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)


In [ ]:
preds = model.predict(test_sequences_matrix)
binary_preds = [1 if item >0.5 else 0 for item in preds]

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_auc_score
accr = model.evaluate(test_sequences_matrix,Y_test)
precision, recall, fscore, _ = precision_recall_fscore_support(Y_test, binary_preds, average='macro')
print("Accuracy:",accr)
print("Precision :",precision)
print("Recall    :", recall)
print("F1 Score  :",fscore)
print("ROC AUC Score:",roc_auc_score(Y_test, preds))

In [ ]:
import pickle

with open('pretrained/lstm_tokenizer.model', 'wb') as handle:
    pickle.dump(tok, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("pretrained/lstm_model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("pretrained/lstm_model.h5")


In [ ]:
test_sequences = tok.texts_to_sequences(["i found a critical vulnerebilirty in an enterprise software"])

test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
model.predict(test_sequences_matrix)